## Problem Overview

The goal of this assignment is to compute an optimal global alignment between two strings using the **Needleman–Wunsch dynamic programming algorithm**.  

Given two strings:

X = x1 x2 ... xm
Y = y1 y2 ... yn

We seek to construct two new strings X_bar and Y_bar of equal length by possibly inserting gaps (-) such that the total alignment penalty is minimized.

Scoring scheme:

Match: 0
Mismatch: 1
Gap: 1

The algorithm fills a dynamic programming table that stores the minimum penalty for aligning all prefixes of the two strings.


## Dynamic Programming Table

To compute the optimal alignment, we define a table \( P \) where each entry represents the minimum penalty for aligning prefixes of the two strings:

$$
P[i][j] = \text{minimum penalty for aligning } X[1..i] \text{ with } Y[1..j]
$$

Here:  
- \( X[1..i] \) is the first \( i \) characters of string \( X \)  
- \( Y[1..j] \) is the first \( j \) characters of string \( Y \)  

The table has dimensions \( (m+1) \times (n+1) \), where \( m \) and \( n \) are the lengths of \( X \) and \( Y \), respectively.  

- **Row 0** represents alignments where \( X \) is empty.  
- **Column 0** represents alignments where \( Y \) is empty.  

These initial rows and columns are filled using the **base cases** (gap penalties) discussed earlier.


## Base Cases

When aligning two strings, the simplest cases occur when one string is empty:

- Aligning an empty string with a string of length \( i \) requires inserting \( i \) gaps.
- Aligning a string of length \( j \) with an empty string requires \( j \) gaps.

These give the base cases for our dynamic programming table \( P \):

$$
P[i][0] = i \cdot g, \quad \text{for } i = 0, 1, \dots, m
$$

$$
P[0][j] = j \cdot g, \quad \text{for } j = 0, 1, \dots, n
$$

Here, \( g \) is the gap penalty, and \( P[i][j] \) represents the optimal alignment score for the first \( i \) characters of string 1 and the first \( j \) characters of string 2.

These initial values fill the first row and first column of the alignment table.


In [3]:

a_gap = 1

def a(x, y):
    """Mismatch / match penalty"""
    if x == y:
        return 0
    return 1


## Recurrence Relation

For each position \( (i, j) \) in the dynamic programming table, we have three possible ways to extend the alignment:

1. **Match or mismatch:** Align \( x_i \) with \( y_j \)  
2. **Insertion:** Align \( x_i \) with a gap in \( Y \)  
3. **Deletion:** Align a gap in \( X \) with \( y_j \)  

The recurrence relation can be written as:

$$
P[i][j] = \min
\begin{cases}
P[i-1][j-1] + a(x_i, y_j), & \text{(match/mismatch)} \\
P[i-1][j] + g, & \text{(insertion)} \\
P[i][j-1] + g, & \text{(deletion)}
\end{cases}
$$

Here:  
- \( a(x_i, y_j) = 0 \) if \( x_i = y_j \) (match), and \( 1 \) if they are different (mismatch)  
- \( g \) is the **gap penalty**  

This recurrence allows us to fill the DP table using previously computed values, eventually giving the **minimum penalty for aligning the full strings**.


## Filling the DP Table

We fill the dynamic programming table **row by row**, using the recurrence relation defined earlier.  

Each cell \( P[i][j] \) depends only on three neighboring cells:

- **Top-left (diagonal):** \( P[i-1][j-1] \), for aligning \( x_i \) with \( y_j \)  
- **Top:** \( P[i-1][j] \), for aligning \( x_i \) with a gap  
- **Left:** \( P[i][j-1] \), for aligning a gap with \( y_j \)  

Because each cell only looks at these three neighbors, the **algorithm runs in \( O(m \cdot n) \) time**, where \( m \) and \( n \) are the lengths of the two strings.  

Once the table is filled, the value in \( P[m][n] \) gives the **minimum penalty for aligning the full strings**.


## Traceback

After filling the dynamic programming table, the **optimal alignment score** is stored in \( P[m][n] \).  

To reconstruct the actual alignment, we perform a **traceback**:

1. Start at the bottom-right cell \( (m, n) \).  
2. At each cell \( (i, j) \), determine which of the three possible previous moves produced the current score:
   - **Diagonal (top-left):** Align \( x_i \) with \( y_j \)  
   - **Top:** Align \( x_i \) with a gap in \( Y \)  
   - **Left:** Align a gap in \( X \) with \( y_j \)  
3. Move to the selected neighboring cell and repeat until reaching \( (0, 0) \).  

This backward path gives the **optimal alignment of the two strings**.


In [5]:
def build_dp_table(X, Y):
    m, n = len(X), len(Y)
    P = [[0]*(n+1) for _ in range(m+1)]

    for i in range(1, m+1):
        P[i][0] = i * a_gap
    for j in range(1, n+1):
        P[0][j] = j * a_gap

    for i in range(1, m+1):
        for j in range(1, n+1):
            P[i][j] = min(
                P[i-1][j-1] + a(X[i-1], Y[j-1]),
                P[i-1][j] + a_gap,
                P[i][j-1] + a_gap
            )
    return P


## Traceback Tie-Breaking Strategy

Multiple optimal alignments may exist with the same minimum score.

To favor alignments with more matching characters, we apply the following priority:

1. Diagonal move **only if characters match**
2. Gap in Y (move up)
3. Gap in X (move left)
4. Diagonal mismatch as last resort

This strategy produces alignments that maximize matches even if extra gaps are introduced.


## Alignment Reconstruction

During traceback:

- A diagonal move appends two characters.
- A vertical move appends a character from X and a gap.
- A horizontal move appends a gap and a character from Y.

Because traceback proceeds from the end of the strings, the constructed alignment is reversed at the end.


## Counting Alignment Statistics

After obtaining the aligned strings, we scan them character by character:

- If either character is `-`, we count a gap.
- If both characters are equal, we count a match.
- Otherwise, we count a mismatch.

These statistics help evaluate alignment quality.


## Complexity Analysis

Let \( m \) and \( n \) be the lengths of the two strings.

- Time Complexity: \( O(mn) \)
- Space Complexity: \( O(mn) \)

This is optimal for global sequence alignment using dynamic programming.


In [6]:
def traceback(P, X, Y):
    i, j = len(X), len(Y)
    AX, AY = [], []

    while i > 0 or j > 0:

        if i > 0 and j > 0 and \
           X[i-1] == Y[j-1] and \
           P[i][j] == P[i-1][j-1] + a(X[i-1], Y[j-1]):

            AX.append(X[i-1])
            AY.append(Y[j-1])
            i -= 1
            j -= 1

        elif i > 0 and P[i][j] == P[i-1][j] + a_gap:
            AX.append(X[i-1])
            AY.append('-')
            i -= 1

        elif j > 0 and P[i][j] == P[i][j-1] + a_gap:
            AX.append('-')
            AY.append(Y[j-1])
            j -= 1

        else:
            AX.append(X[i-1])
            AY.append(Y[j-1])
            i -= 1
            j -= 1

    AX.reverse()
    AY.reverse()
    return ''.join(AX), ''.join(AY)


In [7]:
def count_stats(A, B):
    matches = mismatches = gaps = 0
    for x, y in zip(A, B):
        if x == '-' or y == '-':
            gaps += 1
        elif x == y:
            matches += 1
        else:
            mismatches += 1
    return matches, mismatches, gaps


In [8]:
tests = [
    ["CRANE", "RAIN"],
    ["CYCLE", "BICYCLE"],
    ["ASTRONOMY", "GASTRONOMY"],
    ["INTENTION", "EXECUTION"],
    ["AGGTAB", "GXTXAYB"],
    ["GATTACA", "GCATGCU"],
    ["DELICIOUS", "RELIGIOUS"],
]

for X, Y in tests:
    P = build_dp_table(X, Y)
    AX, AY = traceback(P, X, Y)
    matches, mismatches, gaps = count_stats(AX, AY)

    print(f"{X:<10} -->  |{AX}|    matches: {matches}, mismatches: {mismatches}")
    print(f"{Y:<10} -->  |{AY}|    gaps: {gaps}")
    print()


CRANE      -->  |CRA-NE|    matches: 3, mismatches: 0
RAIN       -->  |-RAIN-|    gaps: 3

CYCLE      -->  |--CYCLE|    matches: 5, mismatches: 0
BICYCLE    -->  |BICYCLE|    gaps: 2

ASTRONOMY  -->  |-ASTRONOMY|    matches: 9, mismatches: 0
GASTRONOMY -->  |GASTRONOMY|    gaps: 1

INTENTION  -->  |INTEN-TION|    matches: 5, mismatches: 3
EXECUTION  -->  |EX-ECUTION|    gaps: 2

AGGTAB     -->  |AGGT-A-B|    matches: 4, mismatches: 1
GXTXAYB    -->  |-GXTXAYB|    gaps: 3

GATTACA    -->  |G-ATTACA|    matches: 4, mismatches: 2
GCATGCU    -->  |GCATG-CU|    gaps: 2

DELICIOUS  -->  |DELICIOUS|    matches: 7, mismatches: 2
RELIGIOUS  -->  |RELIGIOUS|    gaps: 0

